In [1]:
import pandas as pd
import numpy as np

import scipy.sparse
import sklearn.feature_extraction

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm
import platform

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

import os

%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/levabd/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
main_data = np.load('df/main2_data.npy').tolist()
values_data = np.load('df/values2_data.npy').tolist()
order_data = np.load('df/order2_data.npy').tolist()

main_df = pd.DataFrame(main_data)

main_df

important_orders_keys_set = {
    'Upgrade-Insecure-Requests',
    'Accept', 
    'If-Modified-Since',
    'Host', 
    'Connection', 
    'User-Agent', 
    'From', 
    'Accept-Encoding' 
}

important_values_keys_set = {
    'Accept', 
    'Accept-Charset', 
    'Accept-Encoding'
}

from lib.parsers.logParser import LogParser

orders_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
values_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)

l_parser = LogParser(log_folder='Logs/')
l_parser.reassign_orders_values(order_data, values_data)

full_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=True)

 32%|███▏      | 49018/151839 [00:00<00:00, 490172.29it/s]

Sparse dummy orders shape: 
(151839, 48)


100%|██████████| 151839/151839 [00:00<00:00, 339910.99it/s]


Sparse dummy values shape: 
(151839, 283)


In [7]:
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, make_scorer
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing

X_train, X_test, y_train, y_test = train_test_split(full_sparce_dummy, main_df.User_Agent.fillna('NaN'), test_size=0.33, random_state=42)

In [8]:
%%time

from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

CPU times: user 2.5 s, sys: 40 ms, total: 2.54 s
Wall time: 4.68 s


In [13]:
thres = 0.024

In [17]:
predictions_proba = clf.predict_proba(X_test)

In [22]:
predictions_proba[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

# Conclusion

Threshold for DecisionTree not usable because we hawen't probability vector